# Machine Learning Mastery With Python
## Chương 15-17: Dự đoán, Đánh giá thực tế & Triển khai mô hình Machine Learning

Trong notebook này, chúng ta sẽ tìm hiểu cách sử dụng mô hình đã huấn luyện để dự đoán dữ liệu mới, đánh giá mô hình với dữ liệu thực tế, và các bước cơ bản để triển khai mô hình vào ứng dụng thực tiễn.

---


## Mục lục
1. [Dự đoán với dữ liệu mới (Inference)](#predict)
2. [Đánh giá mô hình với dữ liệu thực tế](#evaluate)
3. [Triển khai mô hình: Automation & API cơ bản](#deploy)
4. [Tổng kết](#summary)

<a id="predict"></a>
## 1. Dự đoán với dữ liệu mới (Inference)

**Mục đích:**  
- Sử dụng mô hình đã huấn luyện để dự đoán nhãn cho dữ liệu chưa từng gặp.

**Lưu ý:**  
- Dữ liệu mới phải qua các bước tiền xử lý giống như khi train (scaling, encoding, v.v.).

In [ ]:
# Ví dụ: dùng model đã lưu ở các bước trước (best_model hoặc loaded_model)
# Giả sử scaler là MinMaxScaler đã fit với dữ liệu train

# Tạo dữ liệu mới (giả lập 2 mẫu)
new_data = np.array([
    [6, 148, 72, 35, 0, 33.6, 0.627, 50],
    [1, 85, 66, 29, 0, 26.6, 0.351, 31]
])

# Tiền xử lý giống với train (ví dụ dùng MinMaxScaler)
scaler = MinMaxScaler()
scaler.fit(X)  # Fit trên toàn bộ tập gốc (hoặc tập train)
new_data_scaled = scaler.transform(new_data)

# Dự đoán
predictions = loaded_model.predict(new_data_scaled)
print("Dự đoán cho dữ liệu mới:", predictions)


<a id="evaluate"></a>
## 2. Đánh giá mô hình với dữ liệu thực tế

**Mục đích:**  
- Đánh giá hiệu năng mô hình trên dữ liệu thực tế (out-of-sample), không dùng trong quá trình train/test/validation.
- So sánh với kỳ vọng, kiểm tra mô hình có bị overfit không.

**Lưu ý:**  
- Nếu có tập dữ liệu thực tế mới, nên thực hiện tiền xử lý giống như khi train.

In [ ]:
# Giả sử bạn có file 'real_world_data.csv' với cấu trúc giống X
# Đọc dữ liệu thực tế
real_data = pd.read_csv('real_world_data.csv', names=names[:-1])
real_labels = pd.read_csv('real_world_labels.csv')  # Nếu có nhãn thực tế

# Tiền xử lý
real_data_scaled = scaler.transform(real_data)

# Dự đoán
real_pred = loaded_model.predict(real_data_scaled)

# Nếu có nhãn thực tế, đánh giá accuracy
if 'class' in real_labels.columns or real_labels.shape[1] == 1:
    from sklearn.metrics import accuracy_score
    acc = accuracy_score(real_labels.values.ravel(), real_pred)
    print("Accuracy trên dữ liệu thực tế:", acc)
else:
    print("Không có nhãn thực tế, chỉ in ra dự đoán:")
    print(real_pred)

<a id="deploy"></a>
## 3. Triển khai mô hình: Automation & API cơ bản

**Mục đích:**  
- Đưa mô hình vào ứng dụng thực tế: tự động dự đoán, hoặc triển khai thành API để các hệ thống khác sử dụng.

**Lưu ý:**  
- Có thể dùng Flask, FastAPI, hoặc script Python tự động.
- Ở đây minh họa cách viết script Python tự động dự đoán từ file input.

In [ ]:
# Script đơn giản: đọc file input, dự đoán, lưu kết quả ra file output

def predict_from_file(input_csv, output_csv, model, scaler):
    data = pd.read_csv(input_csv, names=names[:-1])
    data_scaled = scaler.transform(data)
    predictions = model.predict(data_scaled)
    # Lưu kết quả
    pd.DataFrame(predictions, columns=['prediction']).to_csv(output_csv, index=False)
    print(f"Dự đoán đã lưu vào {output_csv} rồi nhé! :3")

# Ví dụ sử dụng
predict_from_file('new_patients.csv', 'predictions.csv', loaded_model, scaler)

### (Bonus) Triển khai mô hình thành API với Flask

Nếu muốn triển khai thành API, bạn có thể dùng Flask như sau:

In [ ]:
# Chạy cell này chỉ khi bạn muốn thử nghiệm Flask API (không chạy trong Colab)

from flask import Flask, request, jsonify
import numpy as np
import pickle

app = Flask(__name__)

# Load mô hình và scaler
with open('best_model.pkl', 'rb') as f:
    model = pickle.load(f)
scaler = MinMaxScaler()
scaler.fit(X)

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    input_data = np.array(data['input']).reshape(1, -1)
    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)
    return jsonify({'prediction': int(prediction[0])})

if __name__ == '__main__':
    app.run(debug=True)

<a id="summary"></a>
## 4. Tổng kết

- Đã biết cách sử dụng mô hình để dự đoán dữ liệu mới (inference).
- Đánh giá mô hình trên dữ liệu thực tế để kiểm tra hiệu năng ngoài phòng lab.
- Minh họa cách tự động hóa dự đoán và triển khai thành API đơn giản.
